In [1]:
import io, time, json
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def read_api_key(filepath):
    """
    Read the Yelp API Key from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """
    
    # feel free to modify this function if you are storing the API Key differently
    with open('api_key.txt', 'r') as f:
        return f.read().replace('\n','')

In [3]:
def all_restaurants(api_key, query):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of dicts representing each business
    """
    
    # Write solution here
    offsetno=0
    total =0
    output=[]
    headers = {        
        "authorization": 'Bearer %s' % api_key, # for the yelp API     
    }
    while (offsetno<=total):
        params = { #parameters are case sensitive!
            "location": query,
            "limit": 20,
            "offset": offsetno,
            "categories": "restaurants"
            
        }
    
        response = requests.get('https://api.yelp.com/v3/businesses/search',
                            headers=headers, params=params)
        result = response.json()
        try:
            output.append( result['businesses'])
        except KeyError:
            print(result)
            return output
        offsetno =offsetno + 20
        total = result['total']
        print (offsetno)
        print (len(output))
    return(output)    

In [4]:
api_key = read_api_key('api_key.txt')
data = all_restaurants(api_key, 'Miami')

20
1
40
2
60
3
80
4
100
5
120
6
140
7
160
8
180
9
200
10
220
11
240
12
260
13
280
14
300
15
320
16
340
17
360
18
380
19
400
20
420
21
440
22
460
23
480
24
500
25
520
26
540
27
560
28
580
29
600
30
620
31
640
32
660
33
680
34
700
35
720
36
740
37
760
38
780
39
800
40
820
41
840
42
860
43
880
44
900
45
920
46
940
47
960
48
980
49
1000
50
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}


In [16]:
new_data = {}
new_data["name"]=[]
new_data["rating"]=[]
new_data["price"]=[]
new_data["review_count"]=[]
new_data["url"]=[]
new_data["category"]=[]
new_data["zip"]=[]
alldata=[]
for i in data:
    for each in i:
        new_data["name"].append(each["name"])
        new_data["rating"].append(each["rating"])
        new_data["url"].append(each["url"])
        try:
            new_data["price"].append(each["price"])
        except KeyError:
            new_data["price"].append(None)

        new_data["review_count"].append(each["review_count"])
        new_data["category"].append(each["categories"][0]["title"])
        new_data["zip"].append(each["location"]["zip_code"])
        alldata.append(each)
    


In [19]:
df = pd.DataFrame(new_data)
df.to_csv("Miami_Yelp.csv",index=False)

In [72]:
df_all = pd.DataFrame(alldata)
df_all.to_csv("Miami_raw.csv",index=False)

In [10]:
data[0][0]["location"]["zip_code"]

'33135'

In [20]:
df = pd.read_excel("/Users/srossgupta/Desktop/Data Focussed Python/HungryGator/DataFiles/Project_prototype_datafiles.xlsx",sheet_name="MergedData")

In [31]:
new_df = df[df["Source"]=="TripAdvisor"]
new_df["check"] = new_df["Address"].apply(lambda x: x[-2:].isdigit() )

TypeError: 'float' object is not subscriptable

In [25]:
b = "963"
b.isdigit()

True

In [34]:
new_df["Address"]

KeyError: 0

In [36]:
df

,Restaurant Name,Url,Rating,Votes,Address,Source,Zip,Unnamed: 7
0,Joe's Stone Crab,https://www.zomato.com/miami/joes-stone-crab-s...,4.5,2821.0,"11 Washington Ave, Miami Beach 33139",Zomato,33139,NaN
1,Versailles Restaurant,https://www.zomato.com/miami/versailles-restau...,4.7,1748.0,"3555 SW 8th Street, Little Havana, Miami 33135",Zomato,33135,NaN
2,La Sandwicherie,https://www.zomato.com/miami/la-sandwicherie-s...,4.6,735.0,"229 14th Street, South Beach, Miami 33139",Zomato,33139,NaN
3,Big Pink,https://www.zomato.com/miami/big-pink-south-be...,4.4,872.0,157 Collins Avenue 33139,Zomato,33139,NaN
4,Yardbird Southern Table & Bar,https://www.zomato.com/miami/yardbird-southern...,4.9,1156.0,"1600 Lenox Avenue, South Beach, Miami 33139",Zomato,33139,NaN
5,Coconuts,https://www.zomato.com/miami/coconuts-fort-lau...,4.5,704.0,"429 Seabreeze Boulevard, Fort Lauderdale 33316",Zomato,33316,NaN
6,SUSHISAMBA,https://www.zomato.com/miami/sushisamba-south-...,4.0,389.0,"600 Lincoln Road, Miami Beach 33139",Zomato,33139,NaN
7,Prime 112,https://www.zomato.com/miami/prime-112-1-south...,4.2,907.0,"112 Ocean Drive, Miami Beach 33139",Zomato,33139,NaN
8,CEVICHE 105,https://www.zomato.com/miami/ceviche-105-downt...,4.5,1054.0,105 Northeast 3rd Avenue 33132,Zomato,33132,NaN
9,Dada,https://www.zomato.com/miami/dada-delray-beach...,4.3,738.0,"52 North Swinton Avenue, Delray Beach 33444",Zomato,33444,NaN
